# Fetch User Anime List
* Given a user, we get an up-to-date version of their anime list
* Supports reading public anime-lists from MyAnimeList and AniList

In [ ]:
import logging
import os

import pandas as pd

In [ ]:
username = ""
source = ""

In [ ]:
allowed_sources = ["MAL", "AniList", "Training"]
assert source in allowed_sources

In [ ]:
data_path = os.path.join("../../data/recommendations", username)
if not os.path.exists(data_path):
    os.mkdir(data_path)

In [ ]:
logger = logging.getLogger("FetchUserAnimeList")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(name)s:%(levelname)s:%(asctime)s: %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
)
for stream in [
    logging.StreamHandler(),
]:
    stream.setFormatter(formatter)
    logger.addHandler(stream)

In [ ]:
print(f"Fetching list for {username} from {source}")

In [ ]:
def import_from_api(username):
    df, ret = get_user_anime_list(username)
    if not ret:
        raise Exception(f"Could not resolve list for {username}")
    return df

In [ ]:
def import_from_mal(username):
    %run ../API/MalApi.ipynb
    return import_from_api(username)

In [ ]:
def import_from_anilist(username):
    %run ../API/AnilistApi.ipynb
    return import_from_api(username)

In [ ]:
def import_from_splits(username, splits):
    userid = int(username)
    dfs = []
    for content in ["explicit", "implicit", "ptw"]:
        for split in splits:
            df = pd.read_csv(f"../../data/splits/{content}_{split}.csv")
            dfs.append(df.loc[lambda x: x["username"] == userid])
    df = pd.concat(dfs, ignore_index=True)
    return df


def import_from_training(username):
    return import_from_splits(username, ["training"])

In [ ]:
import_fns = {
    "MAL": import_from_mal,
    "AniList": import_from_anilist,
    "Training": import_from_training,
}
if not source in import_fns:
    raise Exception(f"Unsupported animelist source {source}")

In [ ]:
df = import_fns[source](username)

In [ ]:
def save_path(source):
    return os.path.join(data_path, f"user_anime_list.{source.lower()}.csv")


for prev_source in allowed_sources:
    if os.path.exists(save_path(prev_source)):
        os.remove(save_path(prev_source))
df.to_csv(save_path(source), index=False)